In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup as bs
import pandas as pd
from time import sleep
import re

In [2]:
url = f"https://www.sec.gov/edgar/search/#/q=dermatitis&dateRange=1y&filter_forms=10-K"

In [3]:
option = webdriver.ChromeOptions()
option.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=option)
driver.get(url)
sleep(5)

In [5]:
content = driver.page_source
content

'<html><head lang="en">\n    <meta charset="UTF-8">\n    <meta http-equiv="X-UA-Compatible" content="IE=11">\n    <meta name="viewport" content="width=device-width, initial-scale=1">\n    \n    <title>SEC.gov | EDGAR Full Text Search</title>\n\n    <!--CSS files-->\n    <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/5.12.0/css/all.min.css">\n    <link rel="stylesheet" href="global/js/jquery-ui/jquery-ui.css" type="text/css">\n    <link rel="stylesheet" href="global/css/bootstrap/bootstrap.min.css" type="text/css">\n    <link rel="stylesheet" href="css/edgar_full_text_search.css">\n    <link rel="stylesheet" href="css/efts_media_queries.css">\n    <link rel="stylesheet" href="css/efts_filters.css" type="text/css">\n    <!-- open source javascript libraries -->\n    <script type="text/javascript" id="www-widgetapi-script" src="https://www.youtube.com/s/player/4e23410d/www-widgetapi.vflset/www-widgetapi.js" async=""></script><script type="text/javascript"

In [6]:
parsed_content = bs(content, 'html.parser')
parsed_content

<html><head lang="en">
<meta charset="utf-8"/>
<meta content="IE=11" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<title>SEC.gov | EDGAR Full Text Search</title>
<!--CSS files-->
<link href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/5.12.0/css/all.min.css" rel="stylesheet"/>
<link href="global/js/jquery-ui/jquery-ui.css" rel="stylesheet" type="text/css"/>
<link href="global/css/bootstrap/bootstrap.min.css" rel="stylesheet" type="text/css"/>
<link href="css/edgar_full_text_search.css" rel="stylesheet"/>
<link href="css/efts_media_queries.css" rel="stylesheet"/>
<link href="css/efts_filters.css" rel="stylesheet" type="text/css"/>
<!-- open source javascript libraries -->
<script async="" id="www-widgetapi-script" src="https://www.youtube.com/s/player/4e23410d/www-widgetapi.vflset/www-widgetapi.js" type="text/javascript"></script><script async="" src="https://www.googletagmanager.com/gtag/js?id=G-CSLL4ZEK4L&amp;l=dataLayer&

In [7]:
hits_div = parsed_content.find("div", attrs={"id":"hits"})
hits_div

<div class="col-md-8 col-lg-9" id="hits">
<div class="mb-4" id="show-result-count" role="alert" style="display: block;">
<h5 style="font-weight: 500;">85 search results</h5>
</div>
<div class="mb-3" id="select-result-columns" style="">
<fieldset style="">
<legend style=""><h6 style="font-weight: bold;">Show Columns</h6></legend>
<div class="form-check form-check-inline show-columns-checkbox" style="">
<input class="form-check-input" id="col-filed" style="" type="checkbox" value="filed"/>
<label class="form-check-label" for="col-filed" style="">Filed</label>
</div>
<div class="form-check form-check-inline show-columns-checkbox" style="">
<input class="form-check-input" id="col-enddate" style="" type="checkbox" value="enddate"/>
<label class="form-check-label" for="col-enddate" style="">Reporting for</label>
</div>
<div class="form-check form-check-inline show-columns-checkbox" style="">
<input class="form-check-input" id="col-cik" style="" type="checkbox" value="cik"/>
<label class="for

In [8]:
table = hits_div.find('table', class_='table')
table

<table class="table" style="">
<thead style="">
<tr style="">
<th class="filetype" id="filetype" style="">Form &amp; File</th>
<th class="filed sortable" style="">Filed </th>
<th class="enddate" style="">Reporting for</th>
<th class="entity-name" style="">Filing entity/person</th>
<th class="cik d-none" style="">CIK</th>
<th class="located d-none" style="">Located</th>
<th class="incorporated d-none" style="">Incorporated</th>
<th class="file-num d-none" style="">File number</th>
<th class="film-num d-none" style="">Film number</th>
</tr>
</thead>
<tbody style=""><tr><td class="filetype"><a class="preview-file" data-adsh="0001787306-24-000031" data-file-name="arqt-20231231.htm" href="#arqt-20231231.htm">10-K (Annual report) </a></td><td class="filed">2024-02-27</td><td class="enddate">2023-12-31</td><td class="entity-name">Arcutis Biotherapeutics, Inc.  (ARQT) </td><td class="cik d-none" nowrap="">CIK 0001787306</td><td class="biz-location located d-none" nowrap="">Westlake Village, CA

In [9]:
rows = table.find('tbody').find_all('tr')
rows

[<tr><td class="filetype"><a class="preview-file" data-adsh="0001787306-24-000031" data-file-name="arqt-20231231.htm" href="#arqt-20231231.htm">10-K (Annual report) </a></td><td class="filed">2024-02-27</td><td class="enddate">2023-12-31</td><td class="entity-name">Arcutis Biotherapeutics, Inc.  (ARQT) </td><td class="cik d-none" nowrap="">CIK 0001787306</td><td class="biz-location located d-none" nowrap="">Westlake Village, CA</td><td class="incorporated d-none" nowrap="">Delaware</td><td class="file-num d-none" nowrap=""><a href="https://www.sec.gov/cgi-bin/browse-edgar/?filenum=001-39186&amp;action=getcompany">001-39186</a></td><td class="film-num d-none" nowrap="">24682024</td></tr>,
 <tr><td class="filetype"><a class="preview-file" data-adsh="0001140361-24-028145" data-file-name="ef20026301_10k.htm" href="#ef20026301_10k.htm">10-K (Annual report) </a></td><td class="filed">2024-05-30</td><td class="enddate">2024-03-31</td><td class="entity-name">Roivant Sciences Ltd.  (ROIV) </td>

In [10]:
len(rows)

85

In [11]:
class_map = {
  'filetype': 'form_and_file',
  'filed': 'filed_on',
  'enddate': 'reporting_for',
  'entity-name': 'filing_entity',
  'cik d-none': 'cik',
  'biz-location located d-none': 'location',
  'incorporated d-none': 'incorporated',
  'file-num d-none': 'file_number',
  'film-num d-none': 'film_number'
}

In [12]:
# df = pd.DataFrame(columns=['form_and_file', 'filed_on', 'reporting_for', 'filing_entity', 'cik', 'location', 'incorporated', 'file_number', 'film_number', 'report_link'])

In [13]:
row_list = []
for row in rows:
  temp = {}
  cols = row.find_all('td')
  
  for col in cols:
    class_name = ' '.join(col.get('class', []))
    text_value = col.get_text(strip=True)
    temp[class_map[class_name]] = text_value
  
  link = row.find('a', class_='preview-file', href=True)
  link_href = link['href']
  link_text = link.text
  xpath = f"//a[@href='{link_href}' and contains(@class, 'preview-file') and text()='{link_text}']"
  
  click_element = driver.find_element(By.XPATH, xpath)
  click_element.click()
  
  iframe_id = 'ipreviewer'
  
  WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.ID, iframe_id))
  )
  
  iframe_content = driver.page_source
  iframe_soup = bs(iframe_content, 'html.parser')
  
  a_tag = iframe_soup.find('a', {'id': 'open-file'})
  temp["report_link"] = a_tag.get("href")
  
  close_button = driver.find_element(By.XPATH, "//button[@class='close' and @data-dismiss='modal']")
  close_button.click()
  
  row_list.append(temp)

In [14]:
driver.close()

In [15]:
df = pd.DataFrame(row_list)

In [16]:
df

,form_and_file,filed_on,reporting_for,filing_entity,cik,location,incorporated,file_number,film_number,report_link
0,10-K (Annual report),2024-02-27,2023-12-31,"Arcutis Biotherapeutics, Inc. (ARQT)",CIK 0001787306,"Westlake Village, CA",Delaware,001-39186,24682024,https://www.sec.gov/Archives/edgar/data/000178...
1,10-K (Annual report),2024-05-30,2024-03-31,Roivant Sciences Ltd. (ROIV),CIK 0001635088,"Hamilton Hm11, D0",Bermuda,001-40782,241006197,https://www.sec.gov/Archives/edgar/data/000163...
2,10-K (Annual report),2024-02-08,2023-12-31,Vanda Pharmaceuticals Inc. (VNDA),CIK 0001347178,"Washington, DC",Delaware,001-34186,24606853,https://www.sec.gov/Archives/edgar/data/000134...
3,10-K (Annual report),2024-02-27,2023-12-31,"Aclaris Therapeutics, Inc. (ACRS)",CIK 0001557746,"Wayne, PA",Delaware,001-37581,24681989,https://www.sec.gov/Archives/edgar/data/000155...
4,10-K (Annual report),2024-03-28,2023-12-31,"STRATA Skin Sciences, Inc. (SSKN)",CIK 0001051514,"Horsham, PA",Delaware,000-51481,24792968,https://www.sec.gov/Archives/edgar/data/000105...
...,...,...,...,...,...,...,...,...,...,...
80,10-K (Annual report),2024-02-29,2023-12-31,"RECURSION PHARMACEUTICALS, INC. (RXRX)",CIK 0001601830,"Salt Lake City, UT",Delaware,001-40323,24699046,https://www.sec.gov/Archives/edgar/data/000160...
81,10-K (Annual report),2024-02-26,2023-12-31,"Day One Biopharmaceuticals, Inc. (DAWN)",CIK 0001845337,"Brisbane, CA",Delaware,001-40431,24679514,https://www.sec.gov/Archives/edgar/data/000184...
82,10-K (Annual report),2024-03-13,2023-12-31,"Aadi Bioscience, Inc. (AADI)",CIK 0001422142,"Pacific Palisades, CA",Delaware,001-38560,24746189,https://www.sec.gov/Archives/edgar/data/000142...
83,10-K (Annual report),2024-04-01,2023-12-31,"Xilio Therapeutics, Inc. (XLO)",CIK 0001840233,"Waltham, MA",Delaware,001-40925,24809603,https://www.sec.gov/Archives/edgar/data/000184...


In [17]:
import requests
from bs4 import BeautifulSoup

# URL of the webpage to scrape
url = "https://www.sec.gov/Archives/edgar/data/1787306/000178730624000031/arqt-20231231.htm"

# Send a GET request to fetch the page content
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36 YourCompanyBot/1.0 (+https://yourcompanywebsite.com)",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    "Accept-Language": "en-US,en;q=0.5",
    "Accept-Encoding": "gzip, deflate, br",
    "Connection": "keep-alive",
    "Upgrade-Insecure-Requests": "1",
    "Referer": "https://www.sec.gov/",
    "DNT": "1",
}

response = requests.get(url, headers=headers)

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')

# Extract all text content
text = soup.get_text(separator='\n').strip()

# Print or use the extracted text
print(text)

arqt-20231231
0001787306
2023
FY
FALSE
P2Y
P2Y
0.25
0001787306
2023-01-01
2023-12-31
0001787306
2023-06-30
iso4217:USD
0001787306
2024-02-21
xbrli:shares
0001787306
2023-10-01
2023-12-31
0001787306
2023-12-31
0001787306
2022-12-31
iso4217:USD
xbrli:shares
0001787306
us-gaap:ProductMember
2023-01-01
2023-12-31
0001787306
us-gaap:ProductMember
2022-01-01
2022-12-31
0001787306
us-gaap:ProductMember
2021-01-01
2021-12-31
0001787306
arqt:OtherRevenueMember
2023-01-01
2023-12-31
0001787306
arqt:OtherRevenueMember
2022-01-01
2022-12-31
0001787306
arqt:OtherRevenueMember
2021-01-01
2021-12-31
0001787306
2022-01-01
2022-12-31
0001787306
2021-01-01
2021-12-31
0001787306
us-gaap:CommonStockMember
2020-12-31
0001787306
us-gaap:AdditionalPaidInCapitalMember
2020-12-31
0001787306
us-gaap:AccumulatedOtherComprehensiveIncomeMember
2020-12-31
0001787306
us-gaap:RetainedEarningsMember
2020-12-31
0001787306
2020-12-31
0001787306
us-gaap:CommonStockMember
2021-01-01
2021-12-31
0001787306
us-gaap:Additiona

In [18]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [19]:
chunk_size = 70000
chunk_overlap = 500

In [20]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
chunks = text_splitter.split_text(text)

In [21]:
len(chunks)

10

In [22]:
from langchain.schema import Document

In [23]:
documents = [Document(page_content=chunk) for chunk in chunks]
documents = [
    Document(
        page_content=chunk,
        metadata={
            "form_and_file": df.iloc[0]["form_and_file"],
            "filed_on": df.iloc[0]["filed_on"],
            "reporting_for": df.iloc[0]["reporting_for"],
            "filing_entity": df.iloc[0]["filing_entity"],
            "cik": df.iloc[0]["cik"],
            "location": df.iloc[0]["location"],
            "incorporated": df.iloc[0]["incorporated"],
            "file_number": df.iloc[0]["file_number"],
            "film_number": df.iloc[0]["film_number"],
            "page_number": i + 1,
            "source": "EDGAR SEC.gov"
        }
    )
    for i, chunk in enumerate(chunks)
]

In [24]:
import configparser
import vertexai

In [25]:
config = configparser.ConfigParser()
config.read('configuration.properties')

project = config['VertexAI']['project']
model_id = config['VertexAI']['model_id']
endpoint_id = config['VertexAI']['endpoint_id']
location = config['VertexAI']['location']
api_endpoint = config['VertexAI']['api_endpoint']
model_pretrained = config['VertexAI']['model_pretrained']

In [26]:
parameters = {
  "max_output_tokens": 1024,
  "temperature": 0.9,
  "top_p": 1
}

In [27]:
import os 
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/home/anshul/Projects/Genassis/edgar-RAG/Notebooks/GCPkey.json" 

In [28]:
vertexai.init(project=project, location=location)

In [29]:
from google.cloud import aiplatform


In [30]:
endpoint = aiplatform.Endpoint(endpoint_id)

In [31]:
instance_dict = {
    "context": '''
    You are analyzing a clinical trial document form the EDGAR sec.gov filings, which will be provided to you in multiple small chunks. 
    
    Each chunk may contain details about different aspects of the trial, including the diseases, drugs, phase, study type, condition being treated, 
    treatment or intervention, sample size, participant demographics, locations, outcome measures, regulatory information, financial information, results, and conclusions. 
    
    Your goal is to: 
    1. Identify the company conducting the trial.
    2. Drugs they are trying to develop.
    3. Diseases they are targeting.
    4. Extract relevant information from each chunk and build a comprehensive summary.
    5. Identifying any additional information to the trial.
    
    If any details in a chunk add to or clarify information from previous chunks, include these updates.
    Add any new findings from the current chunk in the current summary.
    
    The fields you should focus on are:
    
    1. Study Title - The name or official title of the study.
    2. Phase - The phase of the trial, if mentioned (e.g., Phase I, II, III).
    3. Study Type - The type of study (e.g., interventional, observational).
    4. Condition or Disease - The condition or disease the trial is addressing.
    5. Treatment or Intervention - The primary intervention, treatment, or drug under investigation.
    6. Sample Size - Planned or actual number of participants.
    7. Participant Demographics - Details about participant age, sex, or health status, if mentioned.
    8. Locations - List of study sites or countries involved in the trial.
    9. Primary Outcome Measures - The main outcomes that the study aims to assess.
    10. Secondary Outcome Measures - Additional outcomes, if specified.
    11. Results - Key findings or interim data, if available.
    12. Conclusion - Any final conclusions or implications of the trial's findings.

    ''',  
    
    "max_output_tokens": 4096,
    "temperature": 0.9,
    "top_p": 1
}

In [32]:
summaries = []
for document in documents:
  instance_dict["prompt"] = (
        "Extract any relevant clinical trial details from the following text chunk and organize them by the fields below. "
        "If the chunk contains additional details or clarifications for information found in previous chunks, include these updates in your response. "
        "Based on the Information in the current chunk find everything that is important from a clinical trial, financial, regulatory, marketing point of view and include it in the information.\n"
    )
  if summaries != []:
    instance_dict['prompt'] += f"previous chunk information: {summaries}"
    
  instance_dict['prompt'] += f"current chunk {document.page_content}"
  response = endpoint.predict(instances=[instance_dict])
  summaries.append(response.predictions[0]['content'].strip())

In [33]:
summaries

['ZORYVE cream is a novel topical formulation of roflumilast, a highly potent and selective PDE4 inhibitor. PDE4 is a key enzyme in the inflammatory cascade, and its inhibition has been shown to reduce inflammation in a variety of diseases, including psoriasis. In clinical trials, ZORYVE cream demonstrated rapid and significant improvement in psoriasis symptoms, including reductions in plaque thickness, erythema (redness), and scale, as well as significant reductions in pruritus (itching). ZORYVE cream was also well-tolerated, with a low incidence of adverse events, and no serious adverse events were reported. These results suggest that ZORYVE cream may be a valuable new treatment option for patients with plaque psoriasis, particularly those with mild to moderate disease who are seeking a topical treatment with a favorable tolerability profile.',
 "The provided text offers a comprehensive overview of the clinical development, regulatory approvals, and commercialization strategies for Z

In [77]:
from typing import List, Optional
from pydantic import BaseModel, Field

# Define ClinicalTrial as a simple flat structure without optional fields
class ClinicalTrial(BaseModel):
    status: str = Field(description="Status of the trial")
    phase: str = Field(description="Phase of the trial")

# Define Drug with flattened structure and without Optional types
class Drug(BaseModel):
    drug_name: str = Field(description="Name of the drug")
    disease: list[str] = Field(description="Diseases targeted by this drug")
    clinical_trials: List[ClinicalTrial] = Field(description="Details of clinical trials")  # List of ClinicalTrial objects
    regulatory_acceptance: str = Field(description="Regulatory status of the drug")
    financial_details: str = Field(description="Financial details associated with the drug")
    funding_sources: List[str] = Field(description="Funding sources associated with the drug")
    other_details: str = Field(description="Any other details associated with the drug is nothing is there keep it blank")

# Define main model with list of drugs
class DrugReported(BaseModel):
    drugs: List[Drug] = Field(description="List of drugs the company has developed or is developing")



In [78]:
from langchain_google_vertexai import ChatVertexAI

In [79]:
llm = ChatVertexAI(model="gemini-1.5-flash")

In [80]:
structured_llm = llm.with_structured_output(DrugReported)

Key '$defs' is not supported in schema, ignoring


In [81]:
report_prompt = (
  "Provide details about All the different drugs that a pharmaceutical company is developing or has developed,"
  "including the disease targeted, clinical trials, regulatory acceptance, financial details, "
  "funding sources, and risk details for each drug.\n"
  "Summary of companies EDGAR 10K: \n"
)

summary = " ".join(summaries)

report_prompt += summary

In [82]:
temp_res = structured_llm.invoke(report_prompt)

In [83]:
temp_res

DrugReported(drugs=[Drug(drug_name='ZORYVE cream', disease=['Plaque Psoriasis', 'Seborrheic Dermatitis', 'Scalp Psoriasis', 'Atopic Dermatitis'], clinical_trials=[ClinicalTrial(status='Completed', phase='Phase 3'), ClinicalTrial(status='Completed', phase='Phase 2')], regulatory_acceptance='Approved by FDA and EMA', financial_details="ZORYVE cream has generated significant revenue for Arcutis Biotherapeutics, contributing to the company's overall growth. The product's success has driven increased costs of sales but has also led to a decrease in research and development expenses due to the completion of Phase 3 studies.", funding_sources=['Internal Funding', 'Loan Agreement with SLR Investment Corp.'], other_details='ZORYVE cream has received regulatory approvals from the FDA and EMA. The drug has demonstrated promising efficacy and tolerability in clinical trials, showing improvements in plaque thickness, erythema, and pruritus.'), Drug(drug_name='ZORYVE foam', disease=['Plaque Psoriasi

In [84]:
len(temp_res.drugs)

4

In [85]:
for d in temp_res.drugs:
  print(d)

drug_name='ZORYVE cream' disease=['Plaque Psoriasis', 'Seborrheic Dermatitis', 'Scalp Psoriasis', 'Atopic Dermatitis'] clinical_trials=[ClinicalTrial(status='Completed', phase='Phase 3'), ClinicalTrial(status='Completed', phase='Phase 2')] regulatory_acceptance='Approved by FDA and EMA' financial_details="ZORYVE cream has generated significant revenue for Arcutis Biotherapeutics, contributing to the company's overall growth. The product's success has driven increased costs of sales but has also led to a decrease in research and development expenses due to the completion of Phase 3 studies." funding_sources=['Internal Funding', 'Loan Agreement with SLR Investment Corp.'] other_details='ZORYVE cream has received regulatory approvals from the FDA and EMA. The drug has demonstrated promising efficacy and tolerability in clinical trials, showing improvements in plaque thickness, erythema, and pruritus.'
drug_name='ZORYVE foam' disease=['Plaque Psoriasis', 'Seborrheic Dermatitis', 'Scalp Pso

In [87]:
temp_res.drugs

[Drug(drug_name='ZORYVE cream', disease=['Plaque Psoriasis', 'Seborrheic Dermatitis', 'Scalp Psoriasis', 'Atopic Dermatitis'], clinical_trials=[ClinicalTrial(status='Completed', phase='Phase 3'), ClinicalTrial(status='Completed', phase='Phase 2')], regulatory_acceptance='Approved by FDA and EMA', financial_details="ZORYVE cream has generated significant revenue for Arcutis Biotherapeutics, contributing to the company's overall growth. The product's success has driven increased costs of sales but has also led to a decrease in research and development expenses due to the completion of Phase 3 studies.", funding_sources=['Internal Funding', 'Loan Agreement with SLR Investment Corp.'], other_details='ZORYVE cream has received regulatory approvals from the FDA and EMA. The drug has demonstrated promising efficacy and tolerability in clinical trials, showing improvements in plaque thickness, erythema, and pruritus.'),
 Drug(drug_name='ZORYVE foam', disease=['Plaque Psoriasis', 'Seborrheic De